In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

from pathlib import Path
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import json


from IPython.display import display
import pandas_profiling
from pandas_profiling import ProfileReport
%matplotlib inline

from lightfm import LightFM



print(os.path.abspath(''))
root_path = os.path.abspath('')+"/../"
print(root_path)
data_folder = root_path+"/data/"

C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks
C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks/../


C:\Users\samir\anaconda3\envs\retail-rocket\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [ ]:
### read users interaction
events = pd.read_csv(data_folder+"events.csv", header= 0)
events_reduced = events.drop_duplicates()
events_reduced = events_reduced.drop_duplicates(['visitorid', 'itemid'], keep='last').copy()
events_tupel = []
events_final = events_reduced.sort_values(by='timestamp', ascending=True).copy()
events_final.loc[events_final['event'] == 'view', 'rating'] = 2
events_final.loc[events_final['event'] == 'addtocart', 'rating'] = 3
events_final.loc[events_final['event'] == 'transaction', 'rating'] = 5

for index, row in events_final.iterrows():
    events_tupel.append((row['visitorid'], row['itemid'], row['rating']))



In [ ]:
def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()
        

with open(data_folder+'/events_rating_tupel_1.json', 'w') as fp:
    json.dump(events_tupel, fp, default=myconverter, indent=2)

In [ ]:
### load categories and items
category_json = data_folder+"cleaned_category.json"
cleaned_category = None
with open(category_json) as json_file:
    cleaned_category = json.load(json_file)
    
len(cleaned_category) 

In [ ]:
items_json = data_folder+"cleaned_items_indent.json"
cleaned_items = None
with open(items_json) as json_file:
    cleaned_items = json.load(json_file)
    
len(cleaned_items)

In [ ]:
items_array = [] #cleaned_items.values()

for item_id in cleaned_items:
    item_obj = cleaned_items[item_id]
    item = dict()
    categoryid = "_"+str(item_obj['property_categoryid'])
    #print(cleaned_category[categoryid])
    #print(categoryid)
    category = None 
    if categoryid in cleaned_category:
        category = cleaned_category[categoryid]
        #print(category)
    #item = cleaned_items[item_id]
    item['item_id'] = item_id
    item['property_790'] = item_obj['property_790']
    if category is not None:
        paths = category['path'].split('/')
        size = len(paths)
        item['cat_1'] = "_"+paths[0] if size > 0 else None  
        item['cat_2'] = "_"+paths[1] if size > 1 else None
        item['cat_3'] = "_"+paths[2] if size > 2 else None
        
    items_array.append(item)
    #break


print(len(items_array))
#print(items_array[0])
items = pd.DataFrame(items_array)
#items = items[items.columns.drop(list(items.filter(regex='_ts_')))]


In [ ]:
items.head(10)

In [ ]:
events = pd.read_csv(data_folder+"events.csv", header= 0)
events_reduced = events.drop_duplicates()
events_reduced = events_reduced.drop_duplicates(['visitorid', 'event', 'itemid']).copy()
events_final = events_reduced.sort_values(by='timestamp', ascending=True).copy()
events_final.loc[events_final['event'] == 'view', 'rating'] = 2
events_final.loc[events_final['event'] == 'addtocart', 'rating'] = 3
events_final.loc[events_final['event'] == 'transaction', 'rating'] = 5


In [ ]:
events_tupel = []
for index, row in events_final.iterrows():
    events_tupel.append((row['visitorid'], row['itemid'], row['rating']))

In [ ]:
#### buillding user-item matrix
from lightfm.data import Dataset

feature_keys = ['cat_1', 'cat_2', 'cat_3', 'property_790'];

dataset = Dataset(user_identity_features=False, item_identity_features=True)
dataset.fit(events_final['visitorid'].to_numpy(), events_final['itemid'].to_numpy(), item_features=feature_keys)

dataset.interactions_shape()

In [ ]:

start_index = int(0.2*len(events_tupel))
end_index = int(0.25*len(events_tupel))
events_tupel = events_tupel[start_index:end_index]

total_events = len(events_tupel)

train_amt = int(0.9*total_events)
events_train = events_tupel[:train_amt]
events_test = events_tupel[(train_amt+1):]

print(len(events_train))
print(len(events_test))

In [ ]:
(interactions_train, weights_train) = dataset.build_interactions(events_train)
(interactions_test, weights_test) = dataset.build_interactions(events_test)




In [ ]:
#### build features
item_features_tupel = []
items_missing = []
(item_mapping, user_mapping, item_features_mapping, user_features_mapping) = dataset.mapping()
for index, row in items.iterrows():
    item = row['item_id']
    row_features = []
    for attr in feature_keys:
        row_features.append(row[attr])
        
    if item not in item_mapping:
        items_missing.append([item, row_features])
    else:
        item_features_tupel.append([item, row_features])
        
item_features = dataset.build_item_features(item_features_tupel)


In [ ]:

from lightfm import LightFM

model_bpr = LightFM(loss='bpr')
%time model_bpr.fit(interactions_train, item_features=item_features, epochs=4)

In [ ]:

model_wrp = LightFM(loss='warp')
%time model_wrp.fit(interactions_train, item_features=item_features, epochs=4)

In [ ]:
from lightfm.evaluation import auc_score

# Compute and print the AUC score
%time train_auc = auc_score(model_bpr, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:

# Compute and print the AUC score
%time train_auc = auc_score(model_wrp, interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:

%time train_auc = auc_score(model_bpr, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:

%time train_auc = auc_score(model_wrp, interactions_test, train_interactions=interactions_train, item_features=item_features).mean()
print('Collaborative filtering train AUC: %s' % train_auc)